In [7]:
import xarray as xr
import numpy as np
import pandas as pd
import scipy
import time
import scipy.integrate
from scipy.integrate import quad
from scipy.integrate import quad_vec
from scipy.special import gamma

# 3D Wavenumber space
These programs tests the equivalency between the wavenumber space integrals from Feddersen (2007) and Gerbi et al. (2009).<br>
Feddersen's equation is derived from the equation used by Trowbridge and Elgar (2001), but with less restrictive assumptions.<br>
The equation used by Gerbi et al. is more robust for multi-directional waves, and should still yield equivalent values to the other equations.

### Using the old triple nest for loop method


In [10]:
# Check that this integration method is comparable to Fedderson et al. (2007) 
# and Gerbi et al. (2009)
# M_33 from Fedderson should ~ J_33 from Gerbi using certain assumptions

# Assumptions include no current except perpendicular to wave motion
# Best results when vbar = .05, usig = .1
# Best results when vbar and usig are of similar magnitude
ubar = np.sqrt(0**2)
vbar = np.sqrt(0.05**2)
wbar = np.sqrt(0**2)

# Standard deviation in major axis (u) due to unidirectional waves
usig = 0.5
vsig = .01 # Std in v and w directions are set to small value to prevent div by 0 errors
wsig = .01 

uvar = usig**2
vvar = vsig**2
wvar = wsig**2

# Following function is for I(usig/vbar, pi/2)
def I_int(x):
    return (((x**2) - (2 * (vbar/usig) * np.cos(np.pi/2) * x) + ((vbar**2)/(usig**2)))**(1/3)) * np.exp(-.5 * (x**2))
I, I_err = scipy.integrate.quad(I_int, -np.inf, np.inf, epsabs=np.finfo(float).eps, epsrel=np.finfo(float).eps)
I = (1/np.sqrt(2*np.pi)) * (((usig/vbar)**2)**(1/3)) * I

# Use the value of I to calculate M_33(omega) from eq. A8 in Fedderson et al. (2007)
M_33 = (12/55)*((vbar**2)**(1/3))*I
# Omega^5/3 and 2(2pi)^3/2 cancel out if M_ll is plugged into equation A13 from Gerbi et al. (2009)
# Find J_lm using method from Gerbi et al. (2009)

step_dim = 1000

# Create all values of theta, phi, and R within boundaries a to b
R = np.linspace(0, 7, step_dim)
phi = np.linspace(0, 2*np.pi, step_dim)
theta = np.linspace(0, np.pi, step_dim)

# Generate empty array for values of fPhi (dims = [Theta, Phi])
fPhi_33 = np.empty((step_dim,step_dim))

# Empty array to populate with values of fTheta (just 1d to be filled with integral of fPhi)
fTheta_33 = np.empty_like(theta)

# R0, G, and P_lm are all functions of theta and phi, and can be pre-generated as 2d arrays before the nested loop
R0 = np.empty((step_dim,step_dim))
G = np.empty((step_dim,step_dim))
P_33 = np.empty((step_dim,step_dim))

for i in enumerate(theta): # Iterates through theta values (the rows of the empty 2d arrays)
    
    # Each row of the array is filled with the corresponding variable at all values of phi
    R0[i[0]] = ((ubar/usig) * (np.sin(i[1])*np.cos(phi))) + ((vbar/vsig) * (np.sin(i[1])*np.sin(phi)))
    
    G[i[0]] = np.sqrt((np.sin(i[1])**2) * (((np.cos(phi)/usig)**2) + ((np.sin(phi)/vsig)**2)) + ((np.cos(i[1])/wsig)**2))
    
    P_33[i[0]] = ((np.sin(i[1])/G[i[0]])**2) * (((np.cos(phi)/usig)**2) + ((np.sin(phi)/vsig)**2))
    
    # Loop resets and goes through all functions with theta[i+1]
    
# TRIPLE INTEGRAL LOOP
for i in enumerate(theta): # Iterates through theta values only after finding fR[a:b] and fPhi[a:b]
    
    for j in enumerate(phi): # Iterates through each value of phi
        
        # Calculate fR from a to b using pre-generated R0 values
        fR = (R**(2/3))*np.exp(-(((R0[i[0],j[0]]-R)**2)/2))
        
        # Populate empty fPhi array at the given space (theta[i], phi[i])
        fPhi_33[i[0],j[0]] = ((G[i[0],j[0]]**(-11/3))*np.sin(i[1])*P_33[i[0],j[0]] * np.trapz(fR, R))
        
        # Nested loop resets and goes through all functions with phi[i+1], but theta remains constant
    # Populate empty fTheta array with the integral of phi from phi[0] to phi[-1]
    fTheta_33[i[0]] = np.trapz(fPhi_33[i[0]], phi)
    
    if i[0] % 100 == 0: # Progress check every 100 rows
        print(i[0])
        
    # Loop resets and goes through all functions with theta[i+1]

# Evaluate the final integral of fTheta and use it to find J_lm    
J_33 = (1/(2*((2*np.pi)**(3/2))))*(1/(usig*vsig*wsig)) * np.trapz(fTheta_33, theta)

print('M_33: '+str(M_33)+' J_33: '+str(J_33))

C:\Users\Chili\AppData\Local\Temp\ipykernel_5196\1130535519.py:24: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  I, I_err = scipy.integrate.quad(I_int, -np.inf, np.inf, epsabs=np.finfo(float).eps, epsrel=np.finfo(float).eps)


0
100
200
300
400
500
600
700
800
900
M_33: 0.11262710820890497 J_33: 0.1126714492792637


### Using the new vectorized integral method

In [11]:
# Check that this integration method is comparable to Fedderson et al. (2007) 
# and Gerbi et al. (2009)
# M_33 from Fedderson should ~ J_33 from Gerbi using certain assumptions

# Assumptions include no current except perpendicular to wave motion
# Best results when vbar = .05, usig = .1
# Best results when vbar and usig are of similar magnitude
ubar = np.sqrt(0**2)
vbar = np.sqrt(0.05**2)
wbar = np.sqrt(0**2)

# Standard deviation in major axis (u) due to unidirectional waves
usig = 0.5
vsig = .01 # Std in v and w directions are set to small value to prevent div by 0 errors
wsig = .01 

uvar = usig**2
vvar = vsig**2
wvar = wsig**2

# Following function is for I(usig/vbar, pi/2)
def I_int(x):
    return (((x**2) - (2 * (vbar/usig) * np.cos(np.pi/2) * x) + ((vbar**2)/(usig**2)))**(1/3)) * np.exp(-.5 * (x**2))
I, I_err = scipy.integrate.quad(I_int, -np.inf, np.inf, epsabs=np.finfo(float).eps, epsrel=np.finfo(float).eps)
I = (1/np.sqrt(2*np.pi)) * (((usig/vbar)**2)**(1/3)) * I

# Use the value of I to calculate M_33(omega) from eq. A8 in Fedderson et al. (2007)
M_33 = (12/55)*((vbar**2)**(1/3))*I
# Omega^5/3 and 2(2pi)^3/2 cancel out if M_ll is plugged into equation A13 from Gerbi et al. (2009)

# Find J_lm using method from Gerbi et al. (2009)

# Initialize all variables theta, phi, and R within boundaries a to b
phi = np.linspace(0, 2*np.pi, 1000)
theta = np.linspace(0, np.pi, 1000)
thetaRS = np.linspace(0, np.pi, 1000).reshape(1000,1)

fTheta_11 = np.empty(len(theta)) # dims (burst, fPhi)
fTheta_22 = np.empty(len(theta))
fTheta_33 = np.empty(len(theta))

R0 = ((ubar/usig) * (np.sin(thetaRS)*np.cos(phi))) + ((vbar/vsig) * (np.sin(thetaRS)*np.sin(phi)))

G = np.sqrt((np.sin(thetaRS)**2) * (((np.cos(phi)/usig)**2) + ((np.sin(phi)/vsig)**2)) + ((np.cos(thetaRS)/wsig)**2))

P_11 = (1/(G**2))*((((np.sin(thetaRS)**2)*(np.sin(phi)**2))/vvar)+((np.cos(thetaRS)**2)/wvar))
P_22 = (1/(G**2))*((((np.sin(thetaRS)**2)*(np.cos(phi)**2))/uvar)+((np.cos(thetaRS)**2)/wvar))
P_33 = ((np.sin(thetaRS)/G)**2) * (((np.cos(phi)/usig)**2) + ((np.sin(phi)/vsig)**2))

fR = quad_vec(lambda R: (R**(2/3))*np.exp(-(((R0-R)**2)/2)), 0, 7)[0]

fPhi_11 = (G**(-11/3))*np.sin(thetaRS)*P_11 * fR
fPhi_22 = (G**(-11/3))*np.sin(thetaRS)*P_22 * fR
fPhi_33 = (G**(-11/3))*np.sin(thetaRS)*P_33 * fR

for i in enumerate(theta): # Iterates through theta values (the rows of the empty 2d arrays)

    fTheta_11[i[0]] = np.trapz(fPhi_11[i[0]], phi)
    fTheta_22[i[0]] = np.trapz(fPhi_22[i[0]], phi)
    fTheta_33[i[0]] = np.trapz(fPhi_33[i[0]], phi)
    
    if i[0] % 100 == 0: # Progress check every 100 rows
        print(i[0])

# Evaluate the final integral of fTheta and use it to find J_lm    
J_11 = (1/(2*((2*np.pi)**(3/2))))*(1/(usig*vsig*wsig)) * np.trapz(fTheta_11, theta)
J_22 = (1/(2*((2*np.pi)**(3/2))))*(1/(usig*vsig*wsig)) * np.trapz(fTheta_22, theta)
J_33 = (1/(2*((2*np.pi)**(3/2))))*(1/(usig*vsig*wsig)) * np.trapz(fTheta_33, theta)
        
print('M_33: '+str(M_33)+' J_33: '+str(J_33))

C:\Users\Chili\AppData\Local\Temp\ipykernel_5196\1227637627.py:24: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  I, I_err = scipy.integrate.quad(I_int, -np.inf, np.inf, epsabs=np.finfo(float).eps, epsrel=np.finfo(float).eps)


0
100
200
300
400
500
600
700
800
900
M_33: 0.11262710820890497 J_33: 0.11267572917799112


### Testing the validity of Gerbi et al.'s analytical solution
Gerbi et al. use an analytical solution (Eq. A15) to their triple integral that should be near equivalent in value.

In [12]:
# Compare numerical solution of J_ll to analytical solution of M_ll
# From Gerbi et al. (2009) eqs. A13 and A15, respectively

# Assumptions include no current
ubar = np.sqrt(0**2)
vbar = np.sqrt(0**2)
wbar = np.sqrt(0**2)

# Standard deviations u = v = w = sig0
usig = .05 
vsig = .05 
wsig = .05 
sig0 = .05

uvar = usig**2
vvar = vsig**2
wvar = wsig**2

# Initialize all variables theta, phi, and R within boundaries a to b
phi = np.linspace(0, 2*np.pi, 1000)
theta = np.linspace(0, np.pi, 1000)
thetaRS = np.linspace(0, np.pi, 1000).reshape(1000,1)

fTheta_11 = np.empty(len(theta)) # dims (burst, fPhi)
fTheta_22 = np.empty(len(theta))
fTheta_33 = np.empty(len(theta))

R0 = ((ubar/usig) * (np.sin(thetaRS)*np.cos(phi))) + ((vbar/vsig) * (np.sin(thetaRS)*np.sin(phi)))

G = np.sqrt((np.sin(thetaRS)**2) * (((np.cos(phi)/usig)**2) + ((np.sin(phi)/vsig)**2)) + ((np.cos(thetaRS)/wsig)**2))

P_11 = (1/(G**2))*((((np.sin(thetaRS)**2)*(np.sin(phi)**2))/vvar)+((np.cos(thetaRS)**2)/wvar))
P_22 = (1/(G**2))*((((np.sin(thetaRS)**2)*(np.cos(phi)**2))/uvar)+((np.cos(thetaRS)**2)/wvar))
P_33 = ((np.sin(thetaRS)/G)**2) * (((np.cos(phi)/usig)**2) + ((np.sin(phi)/vsig)**2))

fR = quad_vec(lambda R: (R**(2/3))*np.exp(-(((R0-R)**2)/2)), 0, 7)[0]

fPhi_11 = (G**(-11/3))*np.sin(thetaRS)*P_11 * fR
fPhi_22 = (G**(-11/3))*np.sin(thetaRS)*P_22 * fR
fPhi_33 = (G**(-11/3))*np.sin(thetaRS)*P_33 * fR

for i in enumerate(theta): # Iterates through theta values (the rows of the empty 2d arrays)

    fTheta_11[i[0]] = np.trapz(fPhi_11[i[0]], phi)
    fTheta_22[i[0]] = np.trapz(fPhi_22[i[0]], phi)
    fTheta_33[i[0]] = np.trapz(fPhi_33[i[0]], phi)
    
    if i[0] % 100 == 0: # Progress check every 100 rows
        print(i[0])

# Evaluate the final integral of fTheta and use it to find J_lm    
J_11 = (1/(2*((2*np.pi)**(3/2))))*(1/(usig*vsig*wsig)) * np.trapz(fTheta_11, theta)
J_22 = (1/(2*((2*np.pi)**(3/2))))*(1/(usig*vsig*wsig)) * np.trapz(fTheta_22, theta)
J_33 = (1/(2*((2*np.pi)**(3/2))))*(1/(usig*vsig*wsig)) * np.trapz(fTheta_33, theta)

J_ll = J_11+J_22+J_33
M_ll = (((sig0)**(2/3)) * gamma(5/6) * (2**(17/6)) * np.pi)/(2*((2*np.pi)**(3/2)))

print('M_ll: '+str(M_ll)+' J_ll: '+str(J_ll))

0
100
200
300
400
500
600
700
800
900
M_ll: 0.10889979838275736 J_ll: 0.10889970863565548


In [3]:
adv1Despiked = xr.open_dataset('ADV/adv1_despiked.nc')
adv2Despiked = xr.open_dataset('ADV/adv2_despiked.nc')

In [75]:
#Import data
#adv1Int = xr.open_dataset('ADV/adv1_interp.nc')
adv1Patch = xr.open_dataset('ADV/adv1_Patched.nc')
adv1IntAvg = xr.open_dataset('ADV/adv1_IntAvg.nc')

#Import data
#adv2Int = xr.open_dataset('ADV/adv2_interp.nc')
adv2Patch = xr.open_dataset('ADV/adv2_Patched.nc')
adv2IntAvg = xr.open_dataset('ADV/adv2_IntAvg.nc')

In [77]:
def JlmIntegral(data):
    ds = data.copy(deep=True)
    burst_list = np.unique(ds.BurstNum.values)
    J_arr = np.empty((len(burst_list),3))
    
    # Initialize all variables theta, phi, and R within boundaries a to b
    phi = np.linspace(0, 2*np.pi, 1000)
    theta = np.linspace(0, np.pi, 1000)
    thetaRS = np.linspace(0, np.pi, 1000).reshape(1000,1)
    
    fTheta_11 = np.empty(len(theta)) # dims (burst, fPhi)
    fTheta_22 = np.empty(len(theta))
    fTheta_33 = np.empty(len(theta))
        
    for b in enumerate(burst_list):
        
        print('Burst #: '+str(b[1]))
        
        U = ds.Primary.where(ds.BurstNum.isin(b[1]), drop = True).values
        V = ds.Secondary.where(ds.BurstNum.isin(b[1]), drop = True).values
        W = ds.Up.where(ds.BurstNum.isin(b[1]), drop = True).values
    
        #Magnitude of current
        ubar = np.nanmean(np.sqrt(U**2))
        vbar = np.nanmean(np.sqrt(V**2))
        wbar = np.nanmean(np.sqrt(W**2))

        #Standard deviations
        usig = np.nanstd(U)
        vsig = np.nanstd(V) 
        wsig = np.nanstd(W) 
    
        #Variance
        uvar = usig**2
        vvar = vsig**2
        wvar = wsig**2

        # Find J_lm using method from Gerbi et al. (2009)
    
        R0 = ((ubar/usig) * (np.sin(thetaRS)*np.cos(phi))) + ((vbar/vsig) * (np.sin(thetaRS)*np.sin(phi)))
    
        G = np.sqrt((np.sin(thetaRS)**2) * (((np.cos(phi)/usig)**2) + ((np.sin(phi)/vsig)**2)) + ((np.cos(thetaRS)/wsig)**2))
    
        P_11 = (1/(G**2))*((((np.sin(thetaRS)**2)*(np.sin(phi)**2))/vvar)+((np.cos(thetaRS)**2)/wvar))
        P_22 = (1/(G**2))*((((np.sin(thetaRS)**2)*(np.cos(phi)**2))/uvar)+((np.cos(thetaRS)**2)/wvar))
        P_33 = ((np.sin(thetaRS)/G)**2) * (((np.cos(phi)/usig)**2) + ((np.sin(phi)/vsig)**2))
    
        fR = quad_vec(lambda R: (R**(2/3))*np.exp(-(((R0-R)**2)/2)), 0, 7)[0]
    
        fPhi_11 = (G**(-11/3))*np.sin(thetaRS)*P_11 * fR
        fPhi_22 = (G**(-11/3))*np.sin(thetaRS)*P_22 * fR
        fPhi_33 = (G**(-11/3))*np.sin(thetaRS)*P_33 * fR
    
        for i in enumerate(theta): # Iterates through theta values (the rows of the empty 2d arrays)

            fTheta_11[i[0]] = np.trapz(fPhi_11[i[0]], phi)
            fTheta_22[i[0]] = np.trapz(fPhi_22[i[0]], phi)
            fTheta_33[i[0]] = np.trapz(fPhi_33[i[0]], phi)
             
        # Evaluate the final integral of fTheta and use it to find J_lm    
        J_11 = (1/(2*((2*np.pi)**(3/2))))*(1/(usig*vsig*wsig)) * np.trapz(fTheta_11, theta)
        J_22 = (1/(2*((2*np.pi)**(3/2))))*(1/(usig*vsig*wsig)) * np.trapz(fTheta_22, theta)
        J_33 = (1/(2*((2*np.pi)**(3/2))))*(1/(usig*vsig*wsig)) * np.trapz(fTheta_33, theta)
        
        J_arr[b[0]] = [J_11,J_22,J_33]
        
    print('Creating Dataframe')
    JlmDF = pd.DataFrame(J_arr, index = burst_list, columns=['J_11', 'J_22', 'J_33'])
        
    return JlmDF

In [ ]:
Jlm1Int = JlmIntegral(adv1Int)
Jlm1Int.to_csv('ADV/Jlm1Int.csv')
Jlm2Int = JlmIntegral(adv2Int)
Jlm2Int.to_csv('ADV/Jlm2Int.csv')

Jlm1Patch = JlmIntegral(adv1Patch)
Jlm1Patch.to_csv('ADV/Jlm1Patch.csv')
Jlm2Patch = JlmIntegral(adv2Patch)
Jlm2Patch.to_csv('ADV/Jlm2Patch.csv')

Jlm1IntAvg = JlmIntegral(adv1IntAvg)
Jlm1IntAvg.to_csv('ADV/Jlm1IntAvg.csv')
Jlm2IntAvg = JlmIntegral(adv2IntAvg)
Jlm2IntAvg.to_csv('ADV/Jlm2IntAvg.csv')

In [72]:
Jlm1Int

array([0.01766902, 0.01944365, 0.02095513])